# Notebook for cleaning data and generationg output csv

In [87]:
import pandas as pd

In [88]:
# cut and paste sakura data 
# from https://www.data.jma.go.jp/sakura/data/index.html into appropriately named text files
# the following steps process the files
# here we're using the following file
# which contains data gathered on さくら開花, which means sakura flowering

Data notes:

起日の値は月×100＋日で表現される（例：「1231」は12月31日を指す）
date: month*100+day

データの属性(rm)=(6,7,8,9)=（代替種目構内観測、代替種目付近観測、正規種目構内観測、正規種目付近観測）

Data attribute (rm) = (6, 7, 8, 9) = 

(Alternative item campus observation, Alternative item near observation,
Regular item yard observation, Regular item near observation)

In [89]:
import csv
import re
import itertools

def process_line(line):
    # remove odd characters after location
    s_raw = line.replace('\u3000','')
    # match minus following by digit, only replaces minus, sep with comma
    s = re.sub('-(?=\d)', '- ', s_raw) \
        .replace(" ",",") \
        .split(","); 
    return(s)

import csv


def proc_txt(inFile):
    with open(inFile, mode='r') as readFile: 
        lines = readFile.read().splitlines() # read input file and split lines
    with open(inFile+'out.csv',mode='w') as writeFile:
        writer = csv.writer(writeFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        #lines.pop(0) # remove title
        for index,l in enumerate(lines):
            line = process_line(l)
            if len(line)==48:
                writer.writerow(line)
            last_length = len(line)
            if len(line) != last_length:
                print(f'error in length starting at index {index} ')

In [90]:
# creates df grouped by l_code,l_name,year from df with multiple years for columns
def explode_df(df):
    df = df.reset_index(drop=True)
    for i in range(0,df.shape[0]): # loop over rows
        # grab location code and name for each row
        l_code, l_name = df.loc[i,['番号','地点名']]
        # figure out where in clean_data to insert new rows
        if pd.isna(clean_data.index.max()):
            # if empty, insert into first row
            row_index = 0 
        else:
            # insert into row at bottom
            row_index = clean_data.index.max()+1
        # loops over the pairs of columns we want to explode into row entries
        for index, item in enumerate(df.columns):
            next = index + 1
            if ((next < len(df.columns)) & (index > 1) & (index % 2 == 0)):
                # lookup entries for new row from in df
                day = df.iloc[i,index]
                rm = df.iloc[i,next]
                clean_data.loc[row_index,['l_code','l_name','year','day','rm']] = \
                    [l_code,l_name,item,day,rm]
                row_index = row_index + 1
            continue


Initialize empty data frame to hold processed data

In [91]:
clean_data = pd.DataFrame([],columns=['l_code','l_name','year','day','rm'])

In [93]:
# only need to run the proc_txt calls once to generate .txtout files
proc_txt('data/raw/s4_53_1.txt')
proc_txt('data/raw/s4_53_2.txt')
df1 = pd.read_csv('data/raw/s4_53_1.txtout.csv')
df2 = pd.read_csv('data/raw/s4_53_2.txtout.csv')

d1 = pd.concat([df1,df2]); 
d1 = d1.loc[d1['番号'].isnull()==False,:] # sanity check for rows w/missing location
explode_df(d1) # append new data after last existing row in clean_data

Different proc_text function for pages that don't paste nicely like the 56 pages (so all of the other pages, well minus the last one which requires yet another function)

In [94]:
import os
# for the first page of 2016+ text file and associated cols file
# but only round 2 is shown
def process_file_and_cols(filePrefix):
    with open(os.path.join(filePrefix+'.txt'), mode='r') as readFile: 
        lines = readFile.read().splitlines()
    with open(os.path.join(filePrefix+'.txtout.csv'),mode='w') as writeFile:
         writer = csv.writer(writeFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
         #lines.pop(0) # remove title
         for index,l in enumerate(lines):
            line = process_line(l)
            if len(line) > 40:
                line.pop(0)
            writer.writerow(line)
            continue
    with open(os.path.join(filePrefix+'cols.txt'), mode='r') as readFile: 
        lines = readFile.read().splitlines()
    with open(os.path.join(filePrefix+'cols.txtout.csv'),mode='w') as writeFile:
         writer = csv.writer(writeFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
         #lines.pop(0) # remove title
         for index,l in enumerate(lines):
            line = process_line(l)
            if len(line) > 40:
                line.pop(0)
            writer.writerow(line)
            continue

In [95]:
process_file_and_cols('data/raw/s4_76_1')
df1 = pd.read_csv('data/raw/s4_76_1.txtout.csv')
df2 = pd.read_csv('data/raw/s4_76_1cols.txtout.csv')
d1 = pd.concat([df2,df1],axis=1); d1.head()
explode_df(d1)  # append new data after last existing row in clean_data

In [97]:
process_file_and_cols('data/raw/s4_76_2')
df1 = pd.read_csv('data/raw/s4_76_2.txtout.csv')
df2 = pd.read_csv('data/raw/s4_76_2cols.txtout.csv')
d1 = pd.concat([df2,df1],axis=1); d1.head()
explode_df(d1)  # append new data after last existing row in clean_data

In [98]:
clean_data.year.max()

'1995'

In [99]:
process_file_and_cols('data/raw/s4_96_1')
df1 = pd.read_csv('data/raw/s4_96_1.txtout.csv')
df2 = pd.read_csv('data/raw/s4_96_1cols.txtout.csv')
d1 = pd.concat([df2,df1],axis=1); d1.head()
explode_df(d1)  # append new data after last existing row in clean_data

In [100]:
clean_data.year.max()

'2015'

In [101]:
process_file_and_cols('data/raw/s4_96_2')
df1 = pd.read_csv('data/raw/s4_96_2.txtout.csv')
df2 = pd.read_csv('data/raw/s4_96_2cols.txtout.csv')
d1 = pd.concat([df2,df1],axis=1); d1.head()
explode_df(d1)  # append new data after last existing row in clean_data

In [102]:
clean_data.year.max()

'2015'

In [103]:
# for the pages of 2016+ text file and associated cols file
def process_last_pages(filePrefix):
    with open(os.path.join(filePrefix+'.txt'), mode='r') as readFile: 
        lines = readFile.read().splitlines()
    with open(os.path.join(filePrefix+'.txtout.csv'),mode='w') as writeFile:
         writer = csv.writer(writeFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
         #lines.pop(0) # remove title
         for index,l in enumerate(lines):
            line = process_line(l)
            if len(line) > 14:
                line.pop(0)
            writer.writerow(line)
            continue
    with open(os.path.join(filePrefix+'cols.txt'), mode='r') as readFile: 
        lines = readFile.read().splitlines()
    with open(os.path.join(filePrefix+'cols.txtout.csv'),mode='w') as writeFile:
         writer = csv.writer(writeFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
         #lines.pop(0) # remove title
         for index,l in enumerate(lines):
            line = process_line(l)
            if len(line) > 14:
                line.pop(0)
            writer.writerow(line)
            continue

In [104]:
# process first last page
process_last_pages('data/raw/s4_16_1')
df1 = pd.read_csv('data/raw/s4_16_1.txtout.csv').iloc[:,0:6]
df2 = pd.read_csv('data/raw/s4_16_1cols.txtout.csv')
d1 = pd.concat([df2,df1],axis=1);d1.head()
explode_df(d1)  # append new data after last existing row in clean_data

In [105]:
# process second last page
process_last_pages('data/raw/s4_16_2')
df1 = pd.read_csv('data/raw/s4_16_2.txtout.csv').iloc[:,0:6]
df2 = pd.read_csv('data/raw/s4_16_2cols.txtout.csv')
d1 = pd.concat([df2,df1],axis=1);d1.head()
explode_df(d1)  # append new data after last existing row in clean_data

In [106]:
clean_data.loc[clean_data['l_code'].isnull()==True,:] # check for null data by location

,l_code,l_name,year,day,rm


In [107]:
clean_data.drop_duplicates(subset=['l_code','year'],inplace=True)

In [108]:
# some sanity checks to make sure all data is included
clean_data.year.max(), clean_data.year.min()

('2018', '1953')

In [109]:
# check for null values
clean_data.isnull().sum(axis=0)

l_code    0
l_name    0
year      0
day       0
rm        0
dtype: int64

In [110]:
clean_data.to_csv('data/flowering.csv',index=False)

In [111]:
len(clean_data.year.sort_values().unique())

66

In [112]:
clean_data.sort_values('year').head()

,l_code,l_name,year,day,rm
0,401,稚内,1953,521,6
1656,776,洲本,1953,330,8
230,426,浦河,1953,509,6
1633,772,大阪,1953,402,8
1610,770,神戸,1953,404,8
